In [1]:
import torch
from torchtext.vocab import vocab
from torch import nn
import os 
import sys
from gensim.models import Word2Vec
import pandas as pd
from scipy.stats import spearmanr
from box_embeddings.modules.volume.volume import Volume
from box_embeddings.modules.intersection import Intersection
sys.path.append("../")
from utils.model import BoxModel
direc = "/home/fmollica/word2box_pytorch/weights/skipgram_WikiText103/epochs_2_min_count_50_batch_size_32768_embed_dim_2_lr_0.001_window_5_neg_count_5"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(f'{direc}/model_final.pt', map_location=device)
vocab = torch.load(f'{direc}/vocab.pt')
word2vec_model = Word2Vec.load(f'{direc}/word2vec.model')

AllenNLP not available. Registrable won't work.


In [2]:
model

BoxModel(
  (embeddings_word): BoxEmbedding(42038, 4)
  (embeddings_context): BoxEmbedding(42038, 4)
  (box_vol): Volume()
  (box_int): Intersection()
  (vocab): Vocab()
)

In [3]:
def most_similar_2(word, N=None):

        if N is None:
            N = len(model.vocab)

        embedding_all_target = model.embeddings_word.all_boxes

        index_word = (model.vocab[word])

        embedding_word = embedding_all_target[index_word]
                
        sim3 = torch.exp(model.box_vol(model.box_int(embedding_all_target, embedding_word))- torch.maximum(model.box_vol(embedding_word),model.box_vol(embedding_all_target)))

        idx = (-sim3).argsort()

        for i, index in enumerate(idx[0:50]):
            print("Similar to : ", str(index.item()),   model.vocab.lookup_token(index))

In [4]:
from scipy import spatial
def most_similar_2_cosine(word, N=None):

        if N is None:
            N = len(model.vocab)

        embedding_all_target = model.embeddings_word.all_boxes

        index_word = (model.vocab[word])
                
        centroid_word = ((embedding_all_target[index_word].z + embedding_all_target[index_word].Z)/2)       

        centroid_others = ((embedding_all_target.z + embedding_all_target.Z)/2)
        
        cosine = torch.nn.CosineSimilarity(dim=1)

        sim3 = cosine(centroid_word, centroid_others)

        idx = (-sim3).argsort()

        for i, index in enumerate(idx[0:50]):
            print("Similar to : ", str(index.item()),   model.vocab.lookup_token(index))

In [5]:
most_similar_2_cosine("medicine")

Similar to :  32 first
Similar to :  130 portrait
Similar to :  133 voiced
Similar to :  197 expense
Similar to :  235 learn
Similar to :  240 used
Similar to :  257 target
Similar to :  430 new
Similar to :  451 begin
Similar to :  461 look
Similar to :  486 upon
Similar to :  513 removed
Similar to :  525 component
Similar to :  535 needing
Similar to :  649 bonus
Similar to :  650 people
Similar to :  652 transfer
Similar to :  748 richard
Similar to :  921 budget
Similar to :  955 determine
Similar to :  964 massey
Similar to :  987 already
Similar to :  1056 collision
Similar to :  1074 provision
Similar to :  1186 manufacturing
Similar to :  1235 paper
Similar to :  1289 shop
Similar to :  1292 etc
Similar to :  1311 ball
Similar to :  1316 police
Similar to :  1427 furnish
Similar to :  1456 largest
Similar to :  1474 historic
Similar to :  1549 theory
Similar to :  1554 biography
Similar to :  1599 afterward
Similar to :  1649 studio
Similar to :  1687 often
Similar to :  1721 

In [6]:
most_similar_2("medicine")

Similar to :  1204 medicine
Similar to :  25929 mesh
Similar to :  16207 reasonably
Similar to :  12018 consistency
Similar to :  5202 runway
Similar to :  8644 episcopal
Similar to :  20500 mitovich
Similar to :  10011 deadly
Similar to :  13145 retro
Similar to :  23558 outspoken
Similar to :  15798 prescription
Similar to :  6096 uprising
Similar to :  6245 downward
Similar to :  21444 interactivity
Similar to :  11023 sourced
Similar to :  24583 harness
Similar to :  27594 carefree
Similar to :  15262 carlo
Similar to :  18033 muncie
Similar to :  5946 trans
Similar to :  22079 edged
Similar to :  2707 mercury
Similar to :  12257 referee
Similar to :  13283 raider
Similar to :  20012 absorbs
Similar to :  10336 leicestershire
Similar to :  11506 viceroy
Similar to :  12136 collateral
Similar to :  10668 gut
Similar to :  7862 twitter
Similar to :  4967 confessed
Similar to :  16529 slang
Similar to :  1409 forever
Similar to :  10791 wayne
Similar to :  18259 menu
Similar to :  5 b

In [25]:
def save_correlations_results(direc):

    volume = model.box_vol
    intersection = model.box_int

    boxes = model.embeddings_word.all_boxes

    rows_data_int = []
    rows_data_w_int = []
    rows_data_centroids = []

    for filename in os.listdir("/home/fmollica/word2box_pytorch/word_similarity_dataset"):
        dataframe = pd.read_csv("/home/fmollica/word2box_pytorch/word_similarity_dataset/" + filename)
        dataframe.rename( columns={'Unnamed: 0':'index'}, inplace=True )

        list_similarity_word2box_intersection = []
        list_similarity_word2box_weighted_intersection = []
        list_similarity_word2box_centroids = []
        list_similarity_word2vec = []
        list_similarity_sota = []

        if filename!="men.csv":
            for i, row in dataframe.iterrows():
                try:
                    word1_word2vec = word2vec_model.wv.key_to_index[row["word1"]]
                    word2_word2vec = word2vec_model.wv.key_to_index[row["word2"]]
                    word1_word2box = vocab.lookup_indices([row["word1"]])[0]
                    word2_word2box = vocab.lookup_indices([row["word2"]])[0]
                except: 
                    continue
                
                list_similarity_word2vec.append(word2vec_model.wv.similarity(row["word1"], row["word2"]))
                list_similarity_sota.append(row["similarity"])
                #####SIMPLE INTERSECTION BETWEEN BOXES#####
                list_similarity_word2box_intersection.append(torch.exp(volume(intersection(boxes[word1_word2box], boxes[word2_word2box]))).item())
                #####WEIGHTED INTERSECTION BETWEEN BOXES####
                list_similarity_word2box_weighted_intersection.append(torch.exp(volume(intersection(boxes[word1_word2box], boxes[word2_word2box]))-torch.maximum(volume(boxes[word1_word2box]), volume(boxes[word2_word2box]))).item())
                #####COSINE SIMILARITY BETWEEN CENTROIDS#####
                centroid_word1 = ((boxes[word1_word2box].z + boxes[word1_word2box].Z)/2).tolist()
                centroid_word2 = ((boxes[word2_word2box].z + boxes[word2_word2box].Z)/2).tolist()
                list_similarity_word2box_centroids.append(1 - spatial.distance.cosine(centroid_word1, centroid_word2))
                

            rho_vec, p_vec = spearmanr(list_similarity_word2vec, list_similarity_sota)
            rho_box_int, p_box_int = spearmanr(list_similarity_word2box_intersection, list_similarity_sota)
            rho_box_w_int, p_box_w_int = spearmanr(list_similarity_word2box_weighted_intersection, list_similarity_sota)
            rho_box_centroids, p_box_centroids = spearmanr(list_similarity_word2box_centroids, list_similarity_sota)



        else:
            for i, row in dataframe.iterrows():
                try:
                    word1_word2vec = word2vec_model.wv.key_to_index[row["word1"][:-2]]
                    word2_word2vec = word2vec_model.wv.key_to_index[row["word2"][:-2]]
                    word1_word2box = vocab.lookup_indices([row["word1"][:-2]])[0]
                    word2_word2box = vocab.lookup_indices([row["word2"][:-2]])[0]
                except: 
                    continue
                
                list_similarity_word2vec.append(word2vec_model.wv.similarity(row["word1"][:-2], row["word2"][:-2]))
                list_similarity_sota.append(row["similarity"])
                #####SIMPLE INTERSECTION BETWEEN BOXES#####
                list_similarity_word2box_intersection.append(torch.exp(volume(intersection(boxes[word1_word2box], boxes[word2_word2box]))).item())
                #####WEIGHTED INTERSECTION BETWEEN BOXES####
                list_similarity_word2box_weighted_intersection.append(torch.exp(volume(intersection(boxes[word1_word2box], boxes[word2_word2box]))-torch.maximum(volume(boxes[word1_word2box]), volume(boxes[word2_word2box]))).item())
                #####COSINE SIMILARITY BETWEEN CENTROIDS#####
                centroid_word1 = ((boxes[word1_word2box].z + boxes[word1_word2box].Z)/2).tolist()
                centroid_word2 = ((boxes[word2_word2box].z + boxes[word2_word2box].Z)/2).tolist()
                list_similarity_word2box_centroids.append(1 - spatial.distance.cosine(centroid_word1, centroid_word2))

            rho_vec, p_vec = spearmanr(list_similarity_word2vec, list_similarity_sota)
            rho_box_int, p_box_int = spearmanr(list_similarity_word2box_intersection, list_similarity_sota)
            rho_box_w_int, p_box_w_int = spearmanr(list_similarity_word2box_weighted_intersection, list_similarity_sota)
            rho_box_centroids, p_box_centroids = spearmanr(list_similarity_word2box_centroids, list_similarity_sota)

        rows_data_int.append([filename[:-4], rho_vec, p_vec, rho_box_int, p_box_int])
        rows_data_w_int.append([filename[:-4], rho_vec, p_vec, rho_box_w_int, p_box_w_int])
        rows_data_centroids.append([filename[:-4], rho_vec, p_vec, rho_box_centroids, p_box_centroids])

    columns_ = ["dataset", "rho_vec", "p_vec", "rho_box", "p_box"]

    dataf_int = pd.DataFrame(rows_data_int, columns = columns_)
    dataf_w_int = pd.DataFrame(rows_data_w_int, columns = columns_)
    dataf_centroids = pd.DataFrame(rows_data_centroids, columns = columns_)

    dataf_int.to_pickle(direc + "/simple_intersection_correlations.pkl")
    dataf_w_int.to_pickle(direc + "/weighted_intersection_correlations.pkl")
    dataf_centroids.to_pickle(direc + "/centroids_correlations.pkl")



In [26]:
save_correlations_results(os.getcwd())

In [27]:
import pickle
with open(os.getcwd() + '/weighted_intersection_correlations.pkl', 'rb') as f:
    data = pickle.load(f)


data

,dataset,rho_vec,p_vec,rho_box,p_box
0,verb-143,0.248259,8.307688e-03,0.181384,5.562491e-02
1,mturk-771,0.525938,1.652121e-54,0.254416,1.574058e-12
2,rg-65,0.606987,8.313220e-08,0.454312,1.438440e-04
3,men,0.566234,9.024346e-249,0.272187,4.543105e-51
4,wordsim353-rel,0.463134,1.106347e-13,0.146215,2.627094e-02
5,yp-130,0.225059,1.514230e-02,0.192932,3.798899e-02
6,simverb-3500,0.120607,9.722717e-12,0.019849,2.640611e-01
7,mc-30,0.628393,2.004658e-04,0.320125,8.459837e-02
8,simlex999,0.160929,3.826119e-07,0.051939,1.032904e-01
9,mturk-287,0.569230,4.587655e-21,0.324230,5.281393e-07
